# Modélisation 

In [74]:
# Functions imports

import tqdm as notebook_tqdm

import importlib
import functions
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV

importlib.reload(functions)
from functions import *

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [75]:
x = pd.read_csv("../data/processed/x_processed.csv")  # 10605 rows × 10 columns
x_challenge = pd.read_csv("../data/processed/x_challenge_processed.csv")  # 4942 rows × 10 columns
y = pd.read_csv("../data/raw/y_train_jJtXgMX.csv")  # 10605 rows × 2 columns

Préparation des données 

In [76]:
y = y.drop(columns=['DELIVERY_START'], axis=1)
x = x.drop(columns=['DELIVERY_START'], axis=1)


y.head(15)

,spot_id_delta
0,-36.874770
1,-12.643588
2,-1.950193
3,1.938272
4,0.199907
5,-0.943992
6,-2.770881
7,-17.576596
8,-12.352031
9,-15.489499


In [77]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3)

In [78]:
# Normalisation 

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

x_train = pd.DataFrame(x_train) # scaler renvoie des numpy.darray
x_test = pd.DataFrame(x_test)

In [79]:
forest = RandomForestRegressor(
    n_estimators=400,
    min_samples_split=2, 
    min_samples_leaf=1,
    max_features='sqrt',
    max_depth=50,
    bootstrap=False
)

cross_validation(forest, x_train, y_train['spot_id_delta'], 5, 'neg_mean_absolute_percentage_error')

##### CROSS-VALIDATION SCORES ##### 

 neg_mean_absolute_percentage_error  scores from cross-validation  [ -2.21032819  -1.64989797 -17.40634487  -2.18732219  -2.29077894] 
 neg_mean_absolute_percentage_error  mean from cross-validation  -5.148934433868722 
 Standard deviation  6.132910783428426 

 ###################################


In [80]:
# En gros c'est pas mal j'ai juste 6 en std donc je vais la RSCV pour améliorer ça surtout

# Optimisation

In [81]:
# On commence par une Randomized Search, on passera à un HyperOpt plus tard s'il faut car pour l'instant
# j'ai des problèmes avec l'importation d'un package

# Paramètres pour la RandomizedSearchCV

# param_dist = {
#     'n_estimators': [int(x) for x in np.linspace(start=100, stop=1200, num=12)],
#     'max_features': [None, 'sqrt', 'log2'],
#     'max_depth': [int(x) for x in np.linspace(10, 110, num=11)] + [None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'bootstrap': [True, False]
# }

# forest_random = RandomizedSearchCV (
#     estimator=forest,
#     param_distributions=param_dist,
#     n_iter=100,
#     cv=3,
#     verbose=2,
#     n_jobs=-1
# )

# forest_random.fit(x_train, y_train['spot_id_delta'])

# print(f"Meilleurs paramètres : {forest_random.best_params_}")

# Meilleurs paramètres : {'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': False}



Prédiction

In [82]:
forest.fit(x_train, y_train['spot_id_delta'])

# metrics(forest, x_test, y_test, 'Métriques pour la première RandomForest', 'regressor')

RandomForestRegressor(bootstrap=False, max_depth=50, max_features='sqrt',
                      n_estimators=400)

In [83]:
x_challenge = x_challenge.drop(['DELIVERY_START'], axis=1)

y_challenge = forest.predict(x_challenge)

/Users/hugopuybareau/Documents/GitHub/ELMY_DATA_CHALLENGE/.venv/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [84]:
y_challenge = pd.DataFrame({'spot_id_delta' : y_challenge})


y_challenge.head(15)

,spot_id_delta
0,10.766012
1,9.079794
2,20.273424
3,21.175084
4,19.422481
5,19.422481
6,19.422481
7,3.977324
8,5.286654
9,5.220841


In [86]:
y_challenge.to_csv("../data/predicted/y_challenge_1.csv")